<a href="https://colab.research.google.com/github/Ivanrdgz14/Teoria-de-la-informaci-n/blob/main/Canal_de_comunicacion_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
import time
import heapq
from collections import Counter, namedtuple

In [ ]:
def convertir_a_hexadecimal(archivo_entrada):
    try:
        # Abrir el archivo de entrada en modo lectura ('r')
        with open(archivo_entrada, 'r') as entrada:
            # Leer el contenido del archivo
            contenido = entrada.read()

            # Convertir cada carácter en su representación hexadecimal
            contenido_hexadecimal = ''.join([format(ord(char), '02x') for char in contenido])

        # Devolver la cadena en formato hexadecimal
        return contenido_hexadecimal
    except Exception as e:
        # En caso de error, imprimir un mensaje de error y devolver None
        print(f'Error: {str(e)}')
        return None

In [ ]:
# Definición de un nodo para el árbol de Huffman utilizando namedtuple
class NodoHuffman(namedtuple("NodoHuffman", ["caracter", "frecuencia", "izquierda", "derecha"])):
    def __lt__(self, otro):
        return self.frecuencia < otro.frecuencia  # Define una comparación para la cola de prioridad

In [ ]:
def calcular_probabilidad_hex(cadena_hex):
    # Convierte la cadena hexadecimal a una lista de caracteres
    caracteres = [cadena_hex[i:i+2] for i in range(0, len(cadena_hex), 2)]

    # Calcula la frecuencia de cada carácter
    frecuencias = Counter(caracteres)

    # Calcula el número total de caracteres en la cadena
    total_caracteres = len(caracteres)

    # Calcula la probabilidad de cada carácter
    probabilidades = {caracter: frecuencia / total_caracteres for caracter, frecuencia in frecuencias.items()}

    # Ordena las probabilidades en función de las probabilidades en orden descendente
    probabilidades_ordenadas = dict(sorted(probabilidades.items(), key=lambda item: item[1], reverse=True))

    return probabilidades_ordenadas

In [ ]:
# Función para construir el árbol de Huffman a partir de las probabilidades
def construir_arbol_huffman(probabilidades):
    # Inicializa una cola de prioridad con los nodos Hoja (cada carácter es un nodo)
    hojas = [NodoHuffman(caracter, probabilidad, None, None) for caracter, probabilidad in probabilidades.items()]
    heapq.heapify(hojas)  # Convierte la lista en una cola de prioridad

    # Construye el árbol de Huffman combinando nodos Hoja hasta que quede un único nodo Raíz
    while len(hojas) > 1:
        nodo_izquierda = heapq.heappop(hojas)  # Extrae el nodo con menor frecuencia
        nodo_derecha = heapq.heappop(hojas)    # Extrae el siguiente nodo con menor frecuencia
        # Crea un nuevo nodo que tiene como hijos a los dos nodos extraídos y la suma de sus frecuencias
        nodo_padre = NodoHuffman(None, nodo_izquierda.frecuencia + nodo_derecha.frecuencia, nodo_izquierda, nodo_derecha)
        heapq.heappush(hojas, nodo_padre)  # Agrega el nuevo nodo a la cola de prioridad

    # El último nodo en la cola de prioridad es la raíz del árbol de Huffman
    return hojas[0]

In [ ]:
# Función para generar los códigos de Huffman a partir del árbol de Huffman
def generar_codigos_huffman(arbol_huffman):
    def generar_codigos_actual(nodo, codigo_actual=""):
        if nodo is not None:
            if nodo.caracter is not None:
                codigos[nodo.caracter] = codigo_actual  # Asigna el código actual al carácter si es un nodo Hoja
            generar_codigos_actual(nodo.izquierda, codigo_actual + "0")  # Recorre hacia la izquierda (0)
            generar_codigos_actual(nodo.derecha, codigo_actual + "1")    # Recorre hacia la derecha (1)

    codigos = {}  # Diccionario para almacenar los códigos de Huffman
    generar_codigos_actual(arbol_huffman)  # Llama a la función recursiva para generar los códigos
    return codigos

In [ ]:
def shannon_fano(probabilidades):
    # Ordena los caracteres en función de sus probabilidades en orden descendente
    caracteres_ordenados = sorted(probabilidades.items(), key=lambda x: x[1], reverse=True)

    # Crea un diccionario para almacenar los códigos
    codigos = {}

    izquierda = []
    derecha = []

    suma_total = sum(p[1] for p in caracteres_ordenados)
    suma_izquierda = 0
    mitad = suma_total / 2

    for caracter, probabilidad in caracteres_ordenados:
        if suma_izquierda + probabilidad <= mitad:
            izquierda.append((caracter, probabilidad))
            suma_izquierda += probabilidad
        else:
            derecha.append((caracter, probabilidad))

    # Asigna '0' a la parte izquierda y '1' a la parte derecha del árbol
    for caracter, _ in izquierda:
        codigos[caracter] = '0'
    for caracter, _ in derecha:
        codigos[caracter] = '1'

    return codigos


In [40]:
def hexadecimal_a_binario(probabilidades):
    probabilidades_binarias = {}

    for caracter_hex, probabilidad in probabilidades.items():
        caracter_binario = bin(int(caracter_hex, 16))[2:].zfill(8)
        probabilidades_binarias[caracter_hex] = caracter_binario

    return probabilidades_binarias

In [ ]:
def imprimir_codigos(codigos):
    for caracter, codigo in codigos.items():
        print(f"Caracter: {caracter} - Código: {codigo}")

In [41]:
# Definir el nombre del archivo de entrada
archivo_entrada = 'Inteligencia_Artificial.txt'

# Llamar a la función convertir_a_hexadecimal con el nombre del archivo de entrada
contenido_en_hexadecimal = convertir_a_hexadecimal(archivo_entrada)

probabilidades = calcular_probabilidad_hex(contenido_en_hexadecimal)

opcion = int(input("Si quieres shannon_fano escribe 1, Si quieres Huffman escribe 2, Si quieres Binario 3: "))

if opcion == 1:
  print("Elegiste Shannon Fano")
  codigos_shannon = shannon_fano(probabilidades)
  imprimir_codigos(codigos_shannon)

if opcion == 2:

  print("Elegiste Huffman")
  # Construye el árbol de Huffman a partir de las probabilidades
  arbol_huffman = construir_arbol_huffman(probabilidades)

  #  Genera los códigos de Huffman para cada carácter
  codigos_huffman = generar_codigos_huffman(arbol_huffman)
  imprimir_codigos(codigos_huffman)

if opcion == 3:
  print("Elegiste Binario")
  codigo_binario = hexadecimal_a_binario(probabilidades)
  imprimir_codigos(codigo_binario)



Si quieres shannon_fano escribe 1, Si quieres Huffman escribe 2, Si quieres Binario 3: 3
Elegiste Binario
Caracter: 20 - Código: 00100000
Caracter: 61 - Código: 01100001
Caracter: 65 - Código: 01100101
Caracter: 6f - Código: 01101111
Caracter: 73 - Código: 01110011
Caracter: 6e - Código: 01101110
Caracter: 69 - Código: 01101001
Caracter: 72 - Código: 01110010
Caracter: 6c - Código: 01101100
Caracter: 64 - Código: 01100100
Caracter: 74 - Código: 01110100
Caracter: 63 - Código: 01100011
Caracter: 75 - Código: 01110101
Caracter: 6d - Código: 01101101
Caracter: 70 - Código: 01110000
Caracter: 67 - Código: 01100111
Caracter: 76 - Código: 01110110
Caracter: 41 - Código: 01000001
Caracter: 2e - Código: 00101110
Caracter: 79 - Código: 01111001
Caracter: 7a - Código: 01111010
Caracter: 66 - Código: 01100110
Caracter: e1 - Código: 11100001
Caracter: 71 - Código: 01110001
Caracter: 2c - Código: 00101100
Caracter: f3 - Código: 11110011
Caracter: 62 - Código: 01100010
Caracter: 49 - Código: 0100100

In [ ]:
for caracter, probabilidad in probabilidades.items():
    print(f"Carácter: {caracter}, Probabilidad: {probabilidad:.4f}")

Carácter: 20, Probabilidad: 0.1574
Carácter: 61, Probabilidad: 0.1055
Carácter: 65, Probabilidad: 0.0990
Carácter: 6f, Probabilidad: 0.0611
Carácter: 73, Probabilidad: 0.0601
Carácter: 6e, Probabilidad: 0.0578
Carácter: 69, Probabilidad: 0.0542
Carácter: 72, Probabilidad: 0.0469
Carácter: 6c, Probabilidad: 0.0422
Carácter: 64, Probabilidad: 0.0406
Carácter: 74, Probabilidad: 0.0401
Carácter: 63, Probabilidad: 0.0342
Carácter: 75, Probabilidad: 0.0302
Carácter: 6d, Probabilidad: 0.0296
Carácter: 70, Probabilidad: 0.0229
Carácter: 67, Probabilidad: 0.0095
Carácter: 76, Probabilidad: 0.0074
Carácter: 41, Probabilidad: 0.0073
Carácter: 2e, Probabilidad: 0.0069
Carácter: 79, Probabilidad: 0.0069
Carácter: 7a, Probabilidad: 0.0065
Carácter: 66, Probabilidad: 0.0061
Carácter: e1, Probabilidad: 0.0061
Carácter: 71, Probabilidad: 0.0061
Carácter: 2c, Probabilidad: 0.0059
Carácter: f3, Probabilidad: 0.0059
Carácter: 62, Probabilidad: 0.0059
Carácter: 49, Probabilidad: 0.0055
Carácter: 6a, Probab

In [ ]:
def calcular_entropia(cadena_hexadecimal):
    n = 16  # 16 caracteres posibles en hexadecimal (0-9 y A-F)
    frecuencias = {}  # Diccionario para contar las ocurrencias de cada carácter

    # Contar las ocurrencias de cada carácter en la cadena
    for caracter in cadena_hexadecimal:
        frecuencias[caracter] = frecuencias.get(caracter, 0) + 1

    # Calcular la entropía
    entropia = 0.0
    total_caracteres = len(cadena_hexadecimal)
    for frecuencia in frecuencias.values():
        probabilidad = frecuencia / total_caracteres
        entropia -= probabilidad * math.log2(probabilidad)

    return entropia

In [ ]:
# Definición de la función simular_canal_ethernet con retraso selectivo
def simular_canal_ethernet(cadena_hexadecimal, probabilidad_ruido):
    # Inicialización de una cadena vacía para almacenar el resultado con ruido
    cadena_con_ruido = ""

    # Recorremos cada carácter en la cadena hexadecimal de entrada
    for caracter in cadena_hexadecimal:
        # Generar un número aleatorio entre 0 y 1 usando random.random()
        # y compararlo con la probabilidad de ruido para decidir si se aplica ruido o no
        if random.random() < probabilidad_ruido:
            # Si el número aleatorio es menor que la probabilidad de ruido,
            # significa que se activa el ruido y se cambia el carácter a uno aleatorio en hexadecimal
            nuevo_caracter = random.choice("0123456789ABCDEF")
            cadena_con_ruido += nuevo_caracter

            # Agregar un retraso de 0.10 segundos si el carácter fue afectado por el ruido
            time.sleep(0.01)
        else:
            # Si el número aleatorio es igual o mayor que la probabilidad de ruido,
            # se conserva el carácter original sin ruido
            cadena_con_ruido += caracter

    # Devolver la cadena resultante con ruido
    return cadena_con_ruido


In [ ]:
# Definir la probabilidad de ruido como 0.2, lo que significa un 20% de probabilidad de ruido en cada carácter
probabilidad_ruido = 0.2
# Llamar a la función simular_canal_ethernet para aplicar ruido a la cadena hexadecimal original
cadena_con_ruido = simular_canal_ethernet(contenido_en_hexadecimal, probabilidad_ruido)
# Imprimir la cadena hexadecimal original
print("Cadena original:", contenido_en_hexadecimal)
# Imprimir la cadena resultante con ruido
print("Cadena con ruido:", cadena_con_ruido)
# Calcular la entropía del ruido en la cadena con ruido utilizando la función calcular_entropia
entropia_ruido = calcular_entropia(cadena_con_ruido)
# Imprimir la entropía del ruido
print(f'Entropía del ruido: {entropia_ruido}')


Cadena original: 204c6120696e74656c6967656e636961206172746966696369616c202849412920657320756e2063616d706f206465206c6120696e666f726de1746963612071756520736520686120636f6e7665727469646f20656e20756e2074656d612063616e64656e746520656e206c6f7320fa6c74696d6f732061f16f732e204c6120494120736520726566696572652061206c6120636170616369646164206465206c6173206de17175696e61732070617261207265616c697a61722074617265617320717565206e6f726d616c6d656e7465207265717565726972ed616e206c6120696e74656c6967656e6369612068756d616e612e2045737461732074617265617320696e636c7579656e20656c2070726f636573616d69656e746f206465206c656e6775616a65206e61747572616c2c20656c207265636f6e6f63696d69656e746f20646520706174726f6e65732c206c6120746f6d61206465206465636973696f6e65732079206c61207265736f6c756369f36e2064652070726f626c656d61732e204c61204941206861206176616e7a61646f207369676e6966696361746976616d656e746520656e206c617320fa6c74696d61732064e96361646173207920686120656e636f6e747261646f2061706c69636163696f6e657320656e20756e6120616d706c69612

In [ ]:
# Definición de la función restaurar_desde_hexadecimal
def restaurar_desde_hexadecimal(cadena_hexadecimal, archivo_salida):
    try:
        # Abrir el archivo de salida en modo escritura ('w') y especificar la codificación como 'utf-8'
        with open(archivo_salida, 'w', encoding='utf-8') as archivo:
            # Iterar a través de la cadena hexadecimal en bloques de dos caracteres (un byte hexadecimal)
            for i in range(0, len(cadena_hexadecimal), 2):
                hex_byte = cadena_hexadecimal[i:i+2]  # Obtener un byte hexadecimal de dos caracteres
                try:
                    # Convertir el byte hexadecimal en un carácter de texto y escribirlo en el archivo
                    byte = bytes.fromhex(hex_byte).decode('utf-8')
                    archivo.write(byte)
                except ValueError:
                    # En caso de error al convertir los bytes hexadecimales, se escribe un carácter de reemplazo ('??') en el archivo
                    archivo.write('??')

        # Imprimir un mensaje de éxito después de restaurar y escribir el contenido en el archivo
        print(f'Se ha restaurado y escrito el contenido en {archivo_salida}')
    except Exception as e:
        # En caso de error al escribir en el archivo, imprimir un mensaje de error
        print(f'Error al escribir en el archivo: {str(e)}')


In [ ]:
# Asignar la cadena con ruido a la variable cadena_hexadecimal
cadena_hexadecimal = cadena_con_ruido
# Definir el nombre del archivo de salida donde se escribirá el contenido restaurado
archivo_salida = "Inteligencia_Artificial_nuevo"
# Llamar a la función restaurar_desde_hexadecimal para restaurar la cadena_hexadecimal en texto
cadena_texto = restaurar_desde_hexadecimal(cadena_hexadecimal, archivo_salida)



Se ha restaurado y escrito el contenido en Inteligencia_Artificial_nuevo
